In [2]:
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import IPython
import IPython.display as ipd
import soundfile as sf
from pydub import AudioSegment
from pydub import AudioSegment
from pydub.playback import play


%matplotlib inline

In [3]:
def play_audio(filePath):
    """
    input: file path to the audio file. 
    For .wav format, filePath can be used as direct input to waveshow().
    This function plays the audio
    """
    data, sr = librosa.load(filePath, sr=None)
    IPython.core.display.display(ipd.Audio(data=data,rate=sr)) #force the function to render the HTML

def plot_and_play(filePath):
    """
    input: file path to the audio file. 
    For .wav format, filePath can be used as direct input to waveshow().
    This function plots the & displays the audio 
    We would need to force the HTML to be displayed as Audio() is used inside a function
    """
    data, sr = librosa.load(filePath, sr=None)
    ax = plt.figure(figsize=(14, 5))
    plt.axis((None,7,-1,1))

    librosa.display.waveshow(data, sr=sr)
    IPython.core.display.display(ipd.Audio(data=data,rate=sr)) #force the function to render the HTML
    
def flac_to_mp3(filePath):
    sample = AudioSegment.from_file(filePath, format="flac")
    file_handle = sample.export(".".join(filePath.split(".")[:-1]) + ".mp3",
                           format="mp3",)
                           #bitrate="192k",)
def plot_melspectrogram(filePath):
    fig, ax = plt.subplots(figsize=(15,10))
    y, sr = librosa.load(filePath, sr=None)
    S = librosa.feature.melspectrogram(y=y, sr=sr, 
                                       n_mels=128, fmax=8000)
    S_dB = librosa.power_to_db(S, ref=np.max)
    img = librosa.display.specshow(S_dB, x_axis='time',
                                   y_axis='mel', sr=sr,
                                   fmax=8000, ax=ax)
    fig.colorbar(img, ax=ax, format='%+2.0f dB')
    ax.set(title='Mel-frequency spectrogram')
    
def get_duration(filePath, sr=160000):
    y, sr = librosa.load(filePath)
    return librosa.get_duration(y=y,sr=sr)


def cut_and_save_librosa(filePath, start, end, sr=160000):
    """not working"""
    y, sr = librosa.load(filePath, sr=sr)
    y = y[start*sr:end*sr]
    sf.write(".".join(filePath.split(".")[:-1]) + "_cut.wav", y, sr)
    
def cut_duration_pydub(filePath, start, end=None):
    sound = AudioSegment.from_file(filePath, format="wav")
    if end == None:
        new = sound[start*1000:]
    else:
        new = sound[start*1000:end*1000]
    new.export(".".join(filePath.split(".")[:-1]) + "_cut.wav", format="wav")

def remove_duration(filePath, start, end):
    sound = AudioSegment.from_file(filePath, format="wav")
    new = sound[:start*1000] + sound[end*1000:]
    new.export(".".join(filePath.split(".")[:-1]) + "_cut.wav", format="wav")

def merge_files(filePath1, filePath2):
    sound1 = AudioSegment.from_file(filePath1, format="wav")
    sound2 = AudioSegment.from_file(filePath2, format="wav")
    combined = sound1 + sound2
    combined.export(".".join(filePath1.split(".")[:-1]) + "_merged.wav", format="wav")

def merge_all_files(filePathList):
    combined = AudioSegment.empty()
    for filePath in filePathList:
        sound = AudioSegment.from_file(filePath, format="wav")
        combined += sound
    combined.export(".".join(filePathList[0].split(".")[:-1]) + "_merged.wav", format="wav")

def to_mono_librosa(filePath):
    y, sr = librosa.load(filePath, sr=None)
    y_mono = librosa.to_mono(y)
    sf.write(".".join(filePath.split(".")[:-1]) + "_mono.wav", y_mono, sr)

In [73]:
# sound = AudioSegment.from_file("audio/001-051.wav", format="wav")

# new = sound[:1*1000] + sound[3*1000:]
# new.export("test_001-051p.wav", format="wav")

In [82]:
# 001-051p.wav
cut_duration_pydub("audio/001-051p.wav", 0, 0.7)
cut_duration_pydub("audio/001-051_mono.wav", 1.21)
merge_files("audio/001-051p_cut.wav", "audio/001-051_mono_cut.wav")

In [70]:
# 011-014.wav
cut_duration_pydub("audio/011-014p.wav", 0, 1)
cut_duration_pydub("audio/011-014_mono.wav", 1.58)
merge_files("audio/011-014p_cut.wav", "audio/011-014_mono_cut.wav")

In [4]:
# 012-018.wav
cut_duration_pydub("audio/012-018_mono.wav", 0, 3.0) # head

In [6]:
cut_duration_pydub("audio/012-018_mono.wav", 4.3) #tail

In [7]:
cut_duration_pydub("audio/012-018p.wav", 2.5, 3.3)


In [12]:
merge_all_files(["audio/012-018_mono_cut_head.wav", "audio/012-018p_cut.wav", "audio/012-018_mono_cut_tail.wav"])